In [1]:
import sys
import logging, os
from time import time
import numpy as np
import pyvista as pv
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d
%matplotlib notebook

import ipyparallel as ipp

from bem import Electrodes, Sphere, Cylinder, Mesh, Grid, Configuration, Result
from bem.formats import stl
from bem.bemColors_lib.bemColors import bemColors
from helper_functions import helper_functions
from collections import OrderedDict

In [2]:
prefix = "ben_trap2"

In [3]:
scale = 1e-7    # rescale factor 1 um.
use_stl = True

# load electrode faces from colored stl
# s_nta is intermediate processed stl file.
s_nta = stl.read_stl(open("%s.stl" % prefix, "rb"))
print("Import stl:",os.path.abspath("./"+prefix+".stl"),"\n")
print("Electrode colors (numbers):\n")
mesh = Mesh.from_mesh(stl.stl_to_mesh(*s_nta, scale=scale/1e-6,rename={1:"1"})) # unit inside is um

Import stl: C:\Users\Photonics\Documents\Working\Photonics\Ben_Trap\ben_trap2.stl 

Electrode colors (numbers):

dropping 19026
dropping 4634
dropping 10460
dropping 590
dropping 6218
dropping 26892
dropping 14612
dropping 8836
dropping 4436
dropping 27406
dropping 18886
dropping 18444
dropping 4288
dropping 9094
dropping 24912
dropping 27536
dropping 12306
dropping 27078
dropping 9038
dropping 4884
dropping 14848
dropping 342
dropping 4940
dropping 12626
mesh.gather() not working properly, be sure to have valid input


In [4]:
ele_col = bemColors(np.array(list(set(s_nta[2]))),('fusion360','export_stl'))
ele_col.set_my_color(value = (178,178,178),cl_format = ('fusion360','export_stl','RGBA64'),name = 'self_defined')
ele_col.print_stl_colors()

COLORS in the stl:
['_unkColor0']
['_unkColor1']
['_unkColor2']
['_unkColor3']
['_unkColor4']
['_unkColor5']
['_unkColor6']
['_unkColor7']
['_unkColor8']
['_unkColor9']
['_unkColor10']
['_unkColor11']
['_unkColor12']
['_unkColor13']
['_unkColor14']
['_unkColor15']
['_unkColor16']
['_unkColor17']
['_unkColor18']
['_unkColor19']
['_unkColor20']
['_unkColor21']
['_unkColor22']
['_unkColor23']
TOTAL COLORS:  24


In [5]:
# # assign a name for each color
# ele_col.color_electrode(color = 'bem1',name = 'cutout1')
# ele_col.color_electrode(color = 'bem2',name = 'cutout2')
ele_col.color_electrode(color = '_unkColor23',name = 'GND')
ele_col.color_electrode(color = '_unkColor1',name = 'DC1')
ele_col.color_electrode(color = '_unkColor2',name = 'DC2')
ele_col.color_electrode(color = '_unkColor3',name = 'DC3')
ele_col.color_electrode(color = '_unkColor4',name = 'DC4')
ele_col.color_electrode(color = '_unkColor5',name = 'DC5')
ele_col.color_electrode(color = '_unkColor22',name = 'DC6')
ele_col.color_electrode(color = '_unkColor7',name = 'DC7')
ele_col.color_electrode(color = '_unkColor8',name = 'DC8')
ele_col.color_electrode(color = '_unkColor9',name = 'DC9')
ele_col.color_electrode(color = '_unkColor11',name = 'DC10')
ele_col.color_electrode(color = '_unkColor10',name = 'RF')
ele_col.color_electrode(color = '_unkColor12',name = 'DC11')
ele_col.color_electrode(color = '_unkColor13',name = 'DC12')
ele_col.color_electrode(color = '_unkColor14',name = 'DC13')
ele_col.color_electrode(color = '_unkColor15',name = 'DC14')
ele_col.color_electrode(color = '_unkColor16',name = 'DC15')
ele_col.color_electrode(color = '_unkColor17',name = 'DC16')
ele_col.color_electrode(color = '_unkColor18',name = 'DC17')
ele_col.color_electrode(color = '_unkColor0',name = 'DC18')
ele_col.color_electrode(color = '_unkColor20',name = 'DC19')
ele_col.color_electrode(color = '_unkColor21',name = 'DC20')

# # print colors still with no name. These meshes will be neglected in the following codes
ele_col.drop_colors()

# # read stl into mesh with electrode names
# # unnamed meshes will not be imported at all
mesh = Mesh.from_mesh(stl.stl_to_mesh(*s_nta, scale=scale/1e-6,
    rename=ele_col.electrode_colors, quiet=False))

dropping color ['_unkColor6']
dropping color ['_unkColor19']
TOTAL COLORS DROPPED:  2
dropping 19026
1 planes in electrode RF
normals vectors:
 [[2.08166830e-16 1.00000000e+00 3.70075286e-16]]
1 planes in electrode GND
normals vectors:
 [[2.22553301e-16 1.00000000e+00 2.65791236e-16]]
1 planes in electrode DC15
normals vectors:
 [[2.08166764e-16 1.00000000e+00 3.70075286e-16]]
1 planes in electrode DC13
normals vectors:
 [[2.08166764e-16 1.00000000e+00 3.70074703e-16]]
1 planes in electrode DC3
normals vectors:
 [[1.38777891e-16 1.00000000e+00 3.70074412e-16]]
1 planes in electrode DC8
normals vectors:
 [[1.38777891e-16 1.00000000e+00 3.70074253e-16]]
1 planes in electrode DC1
normals vectors:
 [[2.08166764e-16 1.00000000e+00 5.55111142e-16]]
1 planes in electrode DC20
normals vectors:
 [[2.77555783e-16 1.00000000e+00 5.55111512e-16]]
1 planes in electrode DC5
normals vectors:
 [[2.77555783e-16 1.00000000e+00 5.55111512e-16]]
1 planes in electrode DC11
normals vectors:
 [[1.38777891e-1

In [6]:
mesh.plot()

In [7]:
print(len(mesh.points))

710


In [8]:
mesh.triangulate(opts="q20Q",new = False);

start triangulate RF
final opts q20Qzr
finish triangulate RF
start triangulate GND
final opts q20Qzr
finish triangulate GND
start triangulate DC15
final opts q20Qzr
finish triangulate DC15
start triangulate DC13
final opts q20Qzr
finish triangulate DC13
start triangulate DC3
final opts q20Qzr
finish triangulate DC3
start triangulate DC8
final opts q20Qzr
finish triangulate DC8
start triangulate DC1
final opts q20Qzr
finish triangulate DC1
start triangulate DC20
final opts q20Qzr
finish triangulate DC20
start triangulate DC5
final opts q20Qzr
finish triangulate DC5
start triangulate DC11
final opts q20Qzr
finish triangulate DC11
start triangulate DC4
final opts q20Qzr
finish triangulate DC4
start triangulate DC10
final opts q20Qzr
finish triangulate DC10
start triangulate DC2
final opts q20Qzr
finish triangulate DC2
start triangulate DC17
final opts q20Qzr
finish triangulate DC17
start triangulate DC7
final opts q20Qzr
finish triangulate DC7
start triangulate DC12
final opts q20Qzr
fini

In [9]:
mesh.plot()

In [10]:
print(len(mesh.points))

1988


In [11]:
mesh.areas_from_constraints(Sphere(center=np.array([0., 0., 0]),
           radius=5, inside=0.01, outside=1.))    # "inside", "outside" set different mesh densities.
# mesh.areas_from_constraints(Sphere(center=np.array([0., 0., 1]),
#            radius=3.5, inside=0.1, outside=5))    # "inside", "outside" set different mesh densities.
# retriangulate quality and quiet with areas
mesh.triangulate(opts="q20Q", new=False)
# save base mesh to vtk
mesh.to_vtk(prefix)
print("Output vtk:",os.path.abspath("./"+prefix+".vtk"))    # output path


start triangulate RF
final opts q20Qzra
finish triangulate RF
start triangulate GND
final opts q20Qzra
finish triangulate GND
start triangulate DC15
final opts q20Qzra
finish triangulate DC15
start triangulate DC13
final opts q20Qzra
finish triangulate DC13
start triangulate DC3
final opts q20Qzra
finish triangulate DC3
start triangulate DC8
final opts q20Qzra
finish triangulate DC8
start triangulate DC1
final opts q20Qzra
finish triangulate DC1
start triangulate DC20
final opts q20Qzra
finish triangulate DC20
start triangulate DC5
final opts q20Qzra
finish triangulate DC5
start triangulate DC11
final opts q20Qzra
finish triangulate DC11
start triangulate DC4
final opts q20Qzra
finish triangulate DC4
start triangulate DC10
final opts q20Qzra
finish triangulate DC10
start triangulate DC2
final opts q20Qzra
finish triangulate DC2
start triangulate DC17
final opts q20Qzra
finish triangulate DC17
start triangulate DC7
final opts q20Qzra
finish triangulate DC7
start triangulate DC12
final o

In [12]:
mesh.plot()

In [13]:
print(len(mesh.points))

6272


In [36]:
def run_job(args):
    # job is Configuration instance.
    job, grid, prefix = args
    # refine twice adaptively with increasing number of triangles, min angle 15 deg.
#     job.adapt_mesh(triangles=4e2, opts="q0.1Q")
#     job.adapt_mesh(triangles=1e3, opts="q0.1Q")
    # solve for surface charges
    job.solve_singularities(num_mom=4, num_lev=3, max_iter=400)
    # get potentials and fields
    # RF_field = (job.name=="RF1") or (job.name=="RF2") or (job.name=="RF0")
    RF_field = "RF" in job.name
    result = job.simulate(grid, field=RF_field, num_lev=2)    # For "RF", field=True computes the field.
    result.to_vtk(prefix)
    print("finished job %s" % job.name)
    return job.collect_charges()

In [37]:
# grid to evalute potential and fields atCreate a grid in unit of scaled length l. Only choose the interested region (trap center) to save time.
s = 0.005 # step 0.01*100 (scale) = 1 um
Lx, Ly, Lz = 0.6, 0.3, 0.2   # in the unit of scaled length l, z=0 is the bottom of substrate, z = 30um is the top of metal
sx, sy, sz = s, s, s
# ni is grid point number, si is step size. Thus to fix size on i direction you need to fix ni*si.
# nx, ny, nz = [2*np.ceil(L/2.0/s).astype('int')+1 for L in (Lx, Ly, Lz)]
nx = 2*np.ceil(Lx/2.0/sx).astype('int')+1 
ny = 2*np.ceil(Ly/2.0/sy).astype('int')+1 
nz = 2*np.ceil(Lz/2.0/sz).astype('int')+1 
print("Size/l:", Lx, Ly, Lz)
print("Step/l:", sx, sy, sz)
print("Shape (grid point numbers):", nx, ny, nz)
grid = Grid(center=(0, 0, 0), step=(sx, sy, sz), shape=(nx, ny, nz))
# Grid center (nx, ny ,nz)/2 is shifted to origin
print("Grid origin/l:", grid.get_origin()[0])
x, y, z = grid.to_xyz()

Size/l: 0.6 0.3 0.2
Step/l: 0.005 0.005 0.005
Shape (grid point numbers): 121 61 41
Grid origin/l: -0.3


In [38]:
pv.global_theme.notebook = False
def plot_grid_mesh_stl(grid, mesh):
    plotter = pv.Plotter()
    
    # Convert the mesh to PolyData and add it to the plotter
    pd = mesh.to_polydata()
    plotter.add_mesh(pd, scalars='electrode_name', show_edges=True)
    
    # Add the grid points
    x, y, z = np.meshgrid(grid.to_xyz()[0], grid.to_xyz()[1], grid.to_xyz()[2])
    grid_points = np.vstack([x.ravel(), y.ravel(), z.ravel()]).T
    plotter.add_points(grid_points, color='red', point_size=5)
    
    plotter.show_bounds()
    plotter.add_camera_orientation_widget()
    plotter.show()

# Call the function to plot everything together
plot_grid_mesh_stl(grid, mesh)

In [39]:
jobs = list(Configuration.select(mesh, "RF"))    # select() picks one electrode each time.
# jobs = list(Configuration.select(mesh, "RF"))    # select() picks one electrode each time.
# jobs = list(Configuration.select(mesh, "DC.*")) 



# parallel computation
mycluster = ipp.Cluster()
mycluster.start_cluster_sync()
c = mycluster.connect_client_sync()
c.wait_for_engines()

t0 = time()
# Run a parallel map, executing the wrapper function on indices 0,...,n-1
lview = c.load_balanced_view()
# Cause execution on main process to wait while tasks sent to workers finish
lview.block = True 
asyncresult = lview.map_async(run_job, ((job, grid, prefix) for job in jobs))   # Run calculation in parallel
asyncresult.wait_interactive()
print("Computing time: %f s"%(time()-t0))

Starting 32 engines with <class 'ipyparallel.cluster.launcher.LocalEngineSetLauncher'>


  0%|          | 0/32 [00:00<?, ?engine/s]

run_job:   0%|          | 0/1 [00:00<?, ?tasks/s]

Computing time: 218.040998 s


In [40]:
print(len(mesh.points))

6272


In [41]:
import ipyparallel as ipp
from time import time
import psutil

# Select jobs
jobs = list(Configuration.select(mesh, "DC"))    

# Parallel computation setup
mycluster = ipp.Cluster()
mycluster.start_cluster_sync()
c = mycluster.connect_client_sync()
c.wait_for_engines()

# Limit engines to 30
selected_engines = c.ids[:30] if len(c.ids) > 30 else c.ids

# Track memory usage function
def check_memory():
    mem = psutil.virtual_memory()
    print(f"Memory Usage: {mem.percent}% - Available: {mem.available / 1e9:.2f} GB")

# Create a direct view on selected engines
lview = c.direct_view(targets=selected_engines)
lview.block = True

# Run calculation in parallel
t0 = time()
check_memory()

# Run in smaller batches to avoid memory overload
batch_size = min(100, len(jobs))  # Adjust batch size as needed
for i in range(0, len(jobs), batch_size):
    asyncresult = lview.map_async(run_job, ((job, grid, prefix) for job in jobs[i:i + batch_size]))
    asyncresult.wait_interactive()
    check_memory()

print("Computing time: %f s" % (time() - t0))


Starting 32 engines with <class 'ipyparallel.cluster.launcher.LocalEngineSetLauncher'>


  0%|          | 0/32 [00:00<?, ?engine/s]

Memory Usage: 24.0% - Available: 52.06 GB


run_job:   0%|          | 0/20 [00:00<?, ?tasks/s]

Memory Usage: 12.8% - Available: 59.74 GB
Computing time: 130.690303 s


In [22]:
jobs = list(Configuration.select(mesh, "DC"))
for job in jobs:
    run_job((job, grid, prefix))


KeyboardInterrupt



In [19]:
jobs = list(Configuration.select(mesh, "DC"))    # select() picks one electrode each time.
# jobs = list(Configuration.select(mesh, "RF"))    # select() picks one electrode each time.
# jobs = list(Configuration.select(mesh, "DC.*")) 



# parallel computation
mycluster = ipp.Cluster()
mycluster.start_cluster_sync()
c = mycluster.connect_client_sync()
c.wait_for_engines()

t0 = time()
# Run a parallel map, executing the wrapper function on indices 0,...,n-1
lview = c.load_balanced_view()
# Cause execution on main process to wait while tasks sent to workers finish
lview.block = True 
asyncresult = lview.map_async(run_job, ((job, grid, prefix) for job in jobs))   # Run calculation in parallel
asyncresult.wait_interactive()
print("Computing time: %f s"%(time()-t0))

Starting 32 engines with <class 'ipyparallel.cluster.launcher.LocalEngineSetLauncher'>


  0%|          | 0/32 [00:00<?, ?engine/s]

run_job:   0%|          | 0/20 [00:00<?, ?tasks/s]

Failed to remove C:\Users\Photonics\.ipython\profile_default\log\ipengine-1733854763-v8z1-1733854764-31.log: [WinError 32] The process cannot access the file because it is being used by another process: 'C:\\Users\\Photonics\\.ipython\\profile_default\\log\\ipengine-1733854763-v8z1-1733854764-31.log'
Output for 31:
2024-12-10 10:19:28.791 [IPEngine] Loading connection info from $IPP_CONNECTION_INFO
2024-12-10 10:19:28.791 [IPEngine] WARNING | Not using CurveZMQ security
2024-12-10 10:19:28.793 [IPEngine] Registering with controller at tcp://127.0.0.1:55155
2024-12-10 10:19:28.797 [IPEngine] Shell_addrs: ['tcp://127.0.0.1:55157', 'tcp://127.0.0.1:55158', 'tcp://127.0.0.1:55163']
2024-12-10 10:19:28.798 [IPEngine] Connecting shell to tcp://127.0.0.1:55157
2024-12-10 10:19:28.798 [IPEngine] Connecting shell to tcp://127.0.0.1:55158
2024-12-10 10:19:28.798 [IPEngine] Connecting shell to tcp://127.0.0.1:55163
2024-12-10 10:19:28.798 [IPEngine] Starting nanny
2024-12-10 10:19:29.165 [KernelN

In [20]:
Result.view(prefix, 'DC1')

In [21]:
Result.view(prefix, 'RF')

C:\Users\Photonics\.conda\envs\bem39\lib\site-packages\pyvista\core\utilities\misc.py:115: UserWarning: Encountered issue in callback (most recent call last):
  File "C:\Users\Photonics\Documents\Working\bem\bem\electrostatics.py", line 187, in callback_func_iso
    plotter.add_mesh(iso_slider, name = 'iso', cmap = 'Greys', scalars = scalar_name, opacity = 1, show_scalar_bar = False)
  File "C:\Users\Photonics\.conda\envs\bem39\lib\site-packages\pyvista\plotting\plotter.py", line 3438, in add_mesh
    raise ValueError('Empty meshes cannot be plotted. Input mesh has zero points.')
ValueError: Empty meshes cannot be plotted. Input mesh has zero points.
  warnings.warn(formatted_exception)
C:\Users\Photonics\.conda\envs\bem39\lib\site-packages\pyvista\core\utilities\misc.py:115: UserWarning: Encountered issue in callback (most recent call last):
  File "C:\Users\Photonics\Documents\Working\bem\bem\electrostatics.py", line 187, in callback_func_iso
    plotter.add_mesh(iso_slider, name = '

In [42]:
jobs = list(Configuration.select(mesh, "GND"))    # select() picks one electrode each time.
# jobs = list(Configuration.select(mesh, "RF"))    # select() picks one electrode each time.
# jobs = list(Configuration.select(mesh, "DC.*")) 



# parallel computation
mycluster = ipp.Cluster()
mycluster.start_cluster_sync()
c = mycluster.connect_client_sync()
c.wait_for_engines()

t0 = time()
# Run a parallel map, executing the wrapper function on indices 0,...,n-1
lview = c.load_balanced_view()
# Cause execution on main process to wait while tasks sent to workers finish
lview.block = True 
asyncresult = lview.map_async(run_job, ((job, grid, prefix) for job in jobs))   # Run calculation in parallel
asyncresult.wait_interactive()
print("Computing time: %f s"%(time()-t0))

Starting 32 engines with <class 'ipyparallel.cluster.launcher.LocalEngineSetLauncher'>


  0%|          | 0/32 [00:00<?, ?engine/s]

run_job:   0%|          | 0/1 [00:00<?, ?tasks/s]

Computing time: 49.833177 s


In [ ]:
Result.view(prefix, 'GND')